# Use `gpt-4o` to extract data from construction PDFs

## How it works

- Convert each page of the PDF to JPEG (gpt-4o does not support PDFs yet)
- Use multimodal capabilities of `gpt-4o` to extract data from the images
- Enjoy

### Instructions

- Azure OpenAI env variables
- brew install poppler

In [28]:
## Convert pages to JPEG

import os
from rich import print

from pdf2image import convert_from_path

pages = convert_from_path(
    "./data/drawings/Sloping Site - Proposal.pdf",
    500
)

for count, page in enumerate(pages):
    page.save(f'./var/extract_pdf_data_llm/out{count}.jpg', 'JPEG')

In [29]:
def read_image_as_base64(image_path: str) -> str:
    import base64

    with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")

    return image_data

In [30]:
from langchain_openai.chat_models import AzureChatOpenAI
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage


llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT"),
)

In [31]:
## Extract drawing data

class BoundariesSchema(BaseModel): 
    x: int = Field(..., description="X coordinate of the object in mm on the drawing")
    y: int = Field(..., description="Y coordinate of the object in mm on the drawing")
    height: int = Field(..., description="Height of the object in mm on the drawing")
    width: int = Field(..., description="Width of the object in mm on the drawing")

class ObjectSchema(BaseModel): 
    name: str = Field(..., description="Name of the object")
    description: str = Field(..., description="Optional free form details about the object")
    estimated_area: float | None = Field(..., description="Estimated area of the object in square meters")
    boundaries: BoundariesSchema = Field(..., description="Bounding box (x, y, height, width) of the object in mm on the drawing. Assuming drawing is A3")

class OutputSchema(BaseModel): 
    is_drawing: bool = Field(..., description="Detect if input image is drawing")
    title: str = Field(..., description="Title of the drawing")
    name: str = Field(..., description="Name of the drawing")
    date: str = Field(..., description="Date of the drawing")
    notes: str = Field(..., description="Notes of the drawing")
    contact_information: str = Field(..., description="Contact information")
    area: str = Field(..., description="Area information of the drawing in square meters")
    description: str = Field(..., description="Description of the drawing")
    extracted_text: str = Field(..., description="All text for search purposes")
    objects: list[ObjectSchema] = Field(..., description="List of objects depicted on the drawing")


def extract_drawing_info(page_content_base64: str) -> OutputSchema:
    prompt = ChatPromptTemplate.from_messages([
        HumanMessage(
            content=[
                {"type": "text", "text": "Parse drawing information and list of objects on this drawing"},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{page_content_base64}"},
                },
            ],
        ),
    ])
    
    chain = prompt | llm.with_structured_output(OutputSchema)

    return chain.invoke({})

def print_drawing_info(page_path: str):
    page_base64 = read_image_as_base64(page_path)

    print(extract_drawing_info(page_base64))

![Page 4](./var/extract_pdf_data_llm/out4.jpg)

In [32]:
print_drawing_info("./var/extract_pdf_data_llm/out4.jpg")

OutputSchema(
    is_drawing=True,
    title='Floor Plan - Upper',
    name='Outrigger 201',
    date='XXX',
    notes='- Lift off hinges to wc door\n- provide grab rails and other provisions to dw specs\n- Contractor to 
check fit position to be confirmed on site by supervisor\n- Mechanical vents to have back draft natural 
ventilation\n- Referenced vents to be A/S certified\n- Max pressure laminate to 3000mm and over\n- All pull doors 
to be shown in elevation with handle location\n- Hydraulic grab rail position to be marked on basement\n- Utility 
fit off by client after handover\n- Hold out to be fixed top & bottom of sill & side/frame (300mm on higher 
side)\n- where ventilation windows are 1800mm to 2100mm\n- Openable comfort to be between 90% and 9% or greater to 
be restricted\n- Max. 150mm opening sliding windows with key/lever locks\n- awning latch restrictors',
    contact_information='T: (07) 5562 6177 0408 701 623\nW: www.thehousedesigners.com.au\n1/ 7 Strathaird Road 
Bundall QLD 4217\nPO Box 2347 Nerang DC QLD 4210 QBC: 1274038',
    area='201.91 sqm',
    description='This is a detailed architectural floor plan for the upper level, created by The House Designers. 
It includes various rooms, dimensions, and specifications for construction.',
    extracted_text='Note:\n- Lift off hinges to wc door\n- provide grab rails and other provisions to dw specs\n- 
Contractor to check fit position to be confirmed on site by supervisor\n- Mechanical vents to have back draft 
natural ventilation\n- Referenced vents to be A/S certified\n- Max pressure laminate to 3000mm and over\n- All pull
doors to be shown in elevation with handle location\n- Hydraulic grab rail position to be marked on basement\n- 
Utility fit off by client after handover\n- Hold out to be fixed top & bottom of sill & side/frame (300mm on higher
side)\n- where ventilation windows are 1800mm to 2100mm\n- Openable comfort to be between 90% and 9% or greater to 
be restricted\n- Max. 150mm opening sliding windows with key/lever locks\n- awning latch restrictors\n\nT: (07) 
5562 6177 0408 701 623\nW: www.thehousedesigners.com.au\n1/ 7 Strathaird Road Bundall QLD 4217\nPO Box 2347 Nerang 
DC QLD 4210 QBC: 1274038\n\nArea Calculations\nLiving Lower: 29.56 sqm\nLiving Upper: 139.58 sqm\nGarage: 36.08 
sqm\nPorch: 3.98 sqm\nAlfresco: 32.11 sqm\nTotal: 201.91 sqm\n\nPlease Read Carefully\nThe plans and contract to be
used or referred to in the contract & construction of the building works.\nNote: These plans supersede all other 
previous notes or sketches provided or implied whether printed or otherwise.\nOwner: XXX\nDate: XXX\n\nJob No: 
XXX\nDrawing Name: Floor Plan - Upper\nDesign Name: Outrigger 201\nDrawn by: MB\nChecked by: MB\nWorking 
Drawings\n\n450\n70\n2,150\n70\n1,500\n70\n4,100\n70\n2,715\n70\n4,220\n70\n2,700\n70\n900\n3,000\n70\n530\n70\n4,7
85\n70\n1,100\n70\n4,430\n70\n4,930\n15,485\n16,385\n450\n8,960\n70\n1,230\n70\n10,200\n70\n2,920\n70\n8,960\n70\n2
,910\n70\n8,950\n70\n8,920\n70\n8,950\n8,920\n4\n2\n1\n3\n\n',
    objects=[
        ObjectSchema(
            name='Bed 1',
            description='A bedroom with carpet flooring and an ensuite bathroom.',
            estimated_area=15.0,
            boundaries=BoundariesSchema(x=10, y=10, height=100, width=150)
        ),
        ObjectSchema(
            name='Bed 2',
            description='A bedroom with carpet flooring.',
            estimated_area=12.0,
            boundaries=BoundariesSchema(x=170, y=10, height=80, width=150)
        ),
        ObjectSchema(
            name='Bed 3',
            description='A bedroom with carpet flooring.',
            estimated_area=12.0,
            boundaries=BoundariesSchema(x=330, y=10, height=80, width=150)
        ),
        ObjectSchema(
            name='Dining',
            description='A dining area with tile flooring.',
            estimated_area=20.0,
            boundaries=BoundariesSchema(x=10, y=120, height=100, width=200)
        ),
     

In [33]:
## Extract text from PDF

def extract_text_data(page_content_base64: str):
    prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessage(
            content=[
                {"type": "text", "text": "Get text structured and grouped by topics. Output tables as Markdown tables. List as a lists. Format as Markdown"},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{page_content_base64}"},
                },
            ],
        ),
    ]
)
    
    chain = prompt | llm

    return chain.invoke({})

def print_text_data(page_path: str):
    page_base64 = read_image_as_base64(page_path)

    print(extract_text_data(page_base64).content)

![Page 4](./var/extract_pdf_data_llm/out1.jpg)

In [34]:
print_text_data("./var/extract_pdf_data_llm/out1.jpg")

Here is the text structured and grouped by topics:

### General Notes
- Dishwasher provision under sink drainer including cold water connection & GPO
- Ventilation to intruder(s) as per BCA ventilation for each room
- Obscure glass to 2200mm window height to WIP bulkhead over
- All openings to have 2200mm AFL
- All wet areas to have tiling margin set
- NGL: Natural Ground Level
- OF: Out of Heights to Wires, for liveable Balconies
- Dimensions are to be verified prior to commencement of work
- Site dimensions are to have priority to stated dimensions
- All wet areas to have 10mm maximum and 20mm minimum gaps
- All work is to be in accordance with the current Building Code of Australia 3.8.1 & current AS.
- Floor wastes to be 25mm
- Drains: Sewerage Drainer

### Site Notes
- Contractor to verify all boundary clearances & site set out dimensions prior to commencement of construction
- Levels to contours are based on assumed datum. Prior to construction the relevant authority should be contacted 
for possible information on level requirements & flood information
- Retaining wall heights given are the high side (RL) and low side (RL) to be shown where displayed & detailed
- Retaining walls to comply with structural & certification as detailed in the BCA & AS
- Vehicle cross-over to be constructed as per local council requirements & under approval
- Driveways, paths, patios, & floor areas to be pervious surfaces
- Scrape site vegetation to 40% of NGL to provide level building platform
- Finish surface to be graded away from house a minimum of 2:10 for at least 1m
- Site survey & boundary & control points are to be verified prior to commencement of works
- Side entry pathways to be enclosed with current & treated & technical materials

### Stormwater / Drainage Notes
- Drainage to be in accordance with part 3 of the BCA. Refer to the local authority requirements
- All plumbing & drainage is to comply with the standard sewerage layout requirements of the local authority
- All downpipes to be treated in accordance with current Building Code of Australia 3.5.2.5
- Each downpipe must not serve more than 12m of gutter length
- Stormwater outlets to have backflow preventer to 100mm above overflow pipe/box/bubbler to ensure no stormwater 
return
- Roof stormwater to be adjoining property to access to 40mm as per local requirements
- Stormwater approval prior to backfilling to be retained to council policy prior to work commencing
- Discharge points to be located above the inlet to pool containment units, after discharge drainage stormwater to 
remain clean of water

### Energy Efficiency
- To comply with BCA 3.12 unless otherwise stated
- As per Energy Ratings Assessment Certificate

### Construction Notes
| Item                 | Details                                                                 |
|----------------------|-------------------------------------------------------------------------|
| Roofing              | Colorbond steel sheets, thickness as specified on drawings              |
| Ceiling Height       | As specified on drawings                                                |
| Joinery Level        | As specified on drawings                                                |
| Footings             | To engineer's design & detail                                           |
| Slab Height          | To engineer's design & detail                                           |
| Termite Management   | In accordance with BCA & AS3660.2: 313 termite risk                     |
| External Walls - Brick Veneer | 230mm - 110mm brickwork, 50mm Cavity, 70mm Timber Stud        |
| Internal Linings     | 10mm Plasterboard                                                       |
| Ceiling Linings      | 10mm Plasterboard to all wet areas                                      |
| Soffit Linings       | 4.5mm Hardies                                                            |
| Window Flashing      | To all Windows. As per manufacturer’s detail